In [1]:
import pandas as pd
import numpy as np
from numpy import nan as Nan

In [2]:
def convert_county_name(input_county):
    return input_county[:input_county.find(" County")]

def convert_id(input_id):
    return input_id[9:]

In [3]:
county_df = pd.read_csv("datasets/before/2013social.csv").fillna(0) # county info
county_df.columns = county_df.iloc[0] # Set the top row as headers
county_df = county_df[1:]
county_df = county_df[["id", "Geographic Area Name"]]

county_df = county_df.rename(columns={"Geographic Area Name":"name", "id":"FIPS"})
county_df["name"] = county_df["name"].apply(convert_county_name)
county_df["FIPS"] = county_df["FIPS"].apply(convert_id)

county_df = county_df[((county_df["name"] != "Orange") & (county_df["name"] != "San Patricio"))]

county_df.to_csv("datasets/after/county.csv")

# county_df = county_df.set_index("name");

county_df.head()

,FIPS,name
1,48005,Angelina
2,48021,Bastrop
3,48027,Bell
4,48029,Bexar
5,48037,Bowie


In [4]:
def find_county_name(input_num):
    if not (str)(input_num).isdigit():
        return Nan
    
    fips = str(48000 + int(str(input_num)))

    if (county_df["FIPS"] == fips).any():
        return county_df[county_df["FIPS"] == fips].iloc[0]["name"]
    else:
        return Nan

In [5]:
IP_df = []

for year in range(2013, 2020): # Get Hospital Discharges Data from 2013 and 2019
    reader = "datasets/before/" + str(year) + "IP.xlsx"
    IP_original = pd.read_excel(reader).fillna(0)
    IPyear_df = pd.DataFrame()
    for i in range (1, 5): # Get the data of four quarters
        quarter = str(i) + "q" + str(year % 100)
        # print(quarter)
        if year <= 2016:
            qid = quarter + " Inpatient Discharges"
        else:
            qid = "q" + str(i)
            
        if 2014 <= year and year <= 2016:
            county = "Hospital County"
        elif year >= 2017:
            county = "COUNTY"
        else:
            county = "Hospital county"
            
        IPyear_df[[quarter]] = IP_original.loc[:,[county, qid]].groupby(county).sum()
    
    IPyear_df.index.names = ["county"]
    
    if year >= 2017: # Find County name using FIPS
        IPyear_df = IPyear_df.reset_index()
        IPyear_df["name"] = IPyear_df["county"].apply(find_county_name)
        IPyear_df = IPyear_df.dropna().drop(columns=["county"]).set_index("name")
        IPyear_df.index.names = ["county"]
        
    IP_df.append(IPyear_df) #IP_df[i] saves data from Year (2013 + i)

In [6]:
mergedIP_df = IP_df[0] # 2010 data
for year in range(2014, 2020):
    mergedIP_df = pd.concat([mergedIP_df, IP_df[year - 2013]], axis=1, join='inner')

In [7]:
mergedIP_df

,1q13,2q13,3q13,4q13,1q14,2q14,3q14,4q14,1q15,2q15,...,3q17,4q17,1q18,2q18,3q18,4q18,1q19,2q19,3q19,4q19
county,,,,,,,,,,,,,,,,,,,,,
Angelina,3113.0,3440.0,3278.0,3949.0,3244.0,3354.0,3416.0,3456.0,3413,3363,...,3516,3538,3643,3511,3586,3214,3699,3653,3851,3753
Bastrop,98.0,85.0,69.0,85.0,74.0,69.0,55.0,63.0,75,71,...,38,55,52,38,7,39,42,51,37,39
Bell,11955.0,12216.0,12049.0,11872.0,10683.0,8888.0,8237.0,9873.0,9730,16082,...,13065,13445,13268,13111,12781,13247,13020,13009,12828,12687
Bexar,61985.0,61206.0,62147.0,61861.0,62292.0,61904.0,64200.0,64328.0,66555,65078,...,68428,70072,71387,70020,69140,71041,69699,70088,71462,71554
Bowie,6823.0,5986.0,6159.0,5826.0,6025.0,5861.0,6228.0,6260.0,6409,6389,...,6437,6183,6017,5769,5978,5935,6017,6037,6017,5946
Brazoria,1993.0,1866.0,1864.0,1891.0,1913.0,1819.0,1797.0,1904.0,2227,2078,...,3258,2894,4512,3632,3673,3867,3929,3697,3988,3701
Brazos,6915.0,6658.0,6673.0,6540.0,6532.0,6470.0,6716.0,7039.0,6990,7509,...,7187,7143,7679,7464,7455,7533,7158,7483,7519,7714
Cameron,12777.0,11603.0,11434.0,13233.0,9903.0,12951.0,15214.0,12208.0,13048,12479,...,12762,13089,13568,12700,12906,11276,11776,11279,11382,11914
Collin,22589.0,22388.0,22225.0,22337.0,22295.0,24704.0,25464.0,25807.0,25732,26218,...,26304,26960,26867,27187,27069,28096,27886,28206,28369,29614


In [8]:
# Next, we get the percentage
# https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-detail.html
population_df = pd.read_csv("datasets/before/population.csv")
population_df["CTYNAME"] = population_df["CTYNAME"].apply(convert_county_name)
population_df.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,...,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,48,1,Texas,Anderson,1,58458,35521,22937,3135,...,757,898,349,549,865,298,567,39.1,38.8,39.9
1,50,48,1,Texas,Anderson,2,58452,35518,22934,3135,...,758,897,349,548,865,298,567,39.1,38.8,39.9
2,50,48,1,Texas,Anderson,3,58493,35542,22951,3129,...,758,903,358,545,864,301,563,39.1,38.8,39.9
3,50,48,1,Texas,Anderson,4,58394,35586,22808,3105,...,751,916,368,548,842,294,548,39.1,38.7,40.2
4,50,48,1,Texas,Anderson,5,58059,35327,22732,2999,...,780,942,389,553,851,302,549,39.2,38.7,40.5


In [9]:
# Get percentage
for county in mergedIP_df.index:
    for quarter in mergedIP_df.columns:
        year = int(quarter[3:])
        div = population_df.loc[(population_df["CTYNAME"] == county)].loc[:, "POPESTIMATE"].reset_index(drop=True)[year]
        mergedIP_df.at[county, quarter] =  float(mergedIP_df.at[county, quarter]) * (float)(100) / (float)(div)

In [10]:
mergedIP_df.to_csv("datasets/after/mergedIP.csv")
# mergedIP_df

In [11]:
def is_float(input_val):
    try:
        num = float(input_val)
    except ValueError:
        return False
    return True

In [12]:
SOC_df = []

for year in range(2013, 2020): # Get Social Factors from 2010 and 2019
    reader = "datasets/before/" + str(year) + "social.csv"
    SOC_original = pd.read_csv(reader).fillna(0)
    SOC_original.columns = SOC_original.iloc[0] # Set the top row as headers
    SOC_original = SOC_original[1:]

    SOC_original = SOC_original.rename(columns={"Geographic Area Name":"county", "id":"FIPS"})
    SOC_original["county"] = SOC_original["county"].apply(convert_county_name)
    SOC_original["FIPS"] = SOC_original["FIPS"].apply(convert_id)
    SOC_original = SOC_original.set_index("county");
    
    tmp = mergedIP_df[[]]
    for c in SOC_original.columns:
        
        if year == 2018:
            if c.lower()[:18] != "percent estimate!!":
                continue
            else:
                factor = c.lower()[18:]
        else:
            if c.lower()[:9] != "percent!!":
                continue
            else:
                factor = c.lower()[9:]
                
        if (SOC_original[c] == "N").any():
            continue
                
        val = SOC_original.loc["Harrison", c]
        
        if (isinstance(val, pd.Series)) or (not is_float(val)):
            continue
        
        num = float(val)
        if num > 100.0:
            continue
        
        tmp = pd.concat([tmp, SOC_original.loc[:,c]], axis=1, join='inner')
        tmp = tmp.rename(columns={c: factor})
        
    SOC_original = tmp
    SOC_df.append(SOC_original) #SOC_df[i] saves data from Year (2010 + i)

In [13]:
SOC_df[1] # in percentage

,households by type!!total households!!family households (families),households by type!!total households!!family households (families)!!with own children under 18 years,households by type!!total households!!family households (families)!!married-couple family,households by type!!total households!!family households (families)!!married-couple family!!with own children under 18 years,"households by type!!total households!!family households (families)!!male householder, no wife present, family","households by type!!total households!!family households (families)!!male householder, no wife present, family!!with own children under 18 years","households by type!!total households!!family households (families)!!female householder, no husband present, family","households by type!!total households!!family households (families)!!female householder, no husband present, family!!with own children under 18 years",households by type!!total households!!nonfamily households,households by type!!total households!!nonfamily households!!householder living alone,...,place of birth!!total population!!native!!born in united states!!different state,"place of birth!!total population!!native!!born in puerto rico, u.s. island areas, or born abroad to american parent(s)",place of birth!!total population!!foreign born,u.s. citizenship status!!foreign-born population!!naturalized u.s. citizen,u.s. citizenship status!!foreign-born population!!not a u.s. citizen,language spoken at home!!population 5 years and over!!english only,language spoken at home!!population 5 years and over!!language other than english,"language spoken at home!!population 5 years and over!!language other than english!!speak english less than very well""""",computers and internet use!!total households!!with a computer,computers and internet use!!total households!!with a broadband internet subscription
county,,,,,,,,,,,,,,,,,,,,,
Angelina,71.3,34.3,47.3,19.3,6.5,4.3,17.5,10.6,28.7,25.6,...,14.5,0.5,8.0,23.2,76.8,81.8,18.2,6.9,85.1,74.6
Bastrop,74.6,27.4,56.5,17.8,7.0,4.8,11.0,4.8,25.4,21.9,...,19.4,2.3,8.8,25.5,74.5,76.0,24.0,7.6,88.2,77.6
Bell,70.0,35.3,50.3,22.6,5.3,2.9,14.5,9.7,30.0,24.9,...,40.8,5.2,8.3,48.2,51.8,79.1,20.9,5.5,89.8,77.1
Bexar,66.7,31.8,44.5,19.6,5.5,2.5,16.7,9.7,33.3,27.0,...,21.0,2.1,13.2,39.8,60.2,59.6,40.4,12.5,85.0,72.6
Bowie,69.5,28.3,46.9,15.1,5.1,2.9,17.5,10.4,30.5,26.6,...,33.8,0.8,4.9,38.8,61.2,93.4,6.6,3.2,73.2,62.3
Brazoria,76.5,39.1,59.9,29.9,5.8,3.0,10.8,6.1,23.5,19.8,...,21.1,1.0,12.9,53.9,46.1,74.3,25.7,8.5,89.5,78.5
Brazos,52.7,27.1,37.5,17.7,3.3,1.6,11.9,7.7,47.3,28.3,...,20.6,1.3,12.9,24.0,76.0,77.2,22.8,9.1,91.5,76.8
Cameron,79.5,40.9,49.7,23.6,5.7,3.1,24.0,14.2,20.5,18.3,...,9.3,1.1,25.2,30.5,69.5,22.3,77.7,30.2,64.7,52.2
Collin,73.9,40.4,60.7,32.6,3.7,1.9,9.6,5.9,26.1,21.6,...,33.9,1.4,19.7,48.3,51.7,73.3,26.7,9.5,95.9,88.3


In [14]:
for year in range(2013, 2020):
    print(SOC_df[year - 2013].shape)

(51, 61)
(51, 68)
(51, 65)
(51, 68)
(51, 65)
(51, 68)
(51, 65)


In [15]:
factors_df = pd.DataFrame(SOC_df[0].columns).set_index([0])

for year in range(2014, 2020):
    factors_df = pd.concat([factors_df, pd.DataFrame(SOC_df[year - 2013].columns).set_index([0])], axis=1, join='inner')    
    
factors_df.shape

(48, 0)

In [16]:
factors_df = factors_df.reset_index().reset_index()
factors_df.columns = ["label", "name"]

factors_df.to_csv("datasets/after/factors.csv")

In [17]:
factors_df.head() # nLm means label n in Year 20m (e.g. 00L13 means 0th label in Year 2013)

,label,name
0,0,school enrollment!!population 3 years and over...
1,1,school enrollment!!population 3 years and over...
2,2,school enrollment!!population 3 years and over...
3,3,school enrollment!!population 3 years and over...
4,4,school enrollment!!population 3 years and over...


In [18]:
mergedSOC_df = pd.DataFrame()
mergedSOC_df = mergedIP_df[[]]

for year in range(2013, 2020):
    for i in range(0, factors_df.shape[0]):
        if (i < 10):
            label = "0" + str(i) + "L" + str(year % 100)
        else:
            label = str(i) + "L" + str(year % 100)
            
        factor_name = factors_df.at[i, "name"]
        mergedSOC_df = pd.concat([mergedSOC_df, SOC_df[year - 2013].loc[:,factor_name]], axis=1, join='inner')    
        
        #tmp = pd.concat([tmp, SOC_original.loc[:,c]], axis=1, join='inner')
        mergedSOC_df = mergedSOC_df.rename(columns={factor_name: label})

mergedSOC_df.to_csv("datasets/after/mergedSOC.csv")

In [19]:
mergedSOC_df

,00L13,01L13,02L13,03L13,04L13,05L13,06L13,07L13,08L13,09L13,...,38L19,39L19,40L19,41L19,42L19,43L19,44L19,45L19,46L19,47L19
county,,,,,,,,,,,,,,,,,,,,,
Angelina,8.8,6.8,48.6,19.6,16.3,7.4,14.5,29.6,26.6,6.1,...,28.7,55.5,1.3,3.6,10.9,23.0,51.3,2.9,6.8,16.0
Bastrop,3.1,8.3,50.9,20.1,17.6,10.3,10.1,31.0,24.5,6.6,...,36.0,47.3,1.1,2.2,13.5,27.4,43.3,1.0,11.3,17.0
Bell,5.6,5.5,41.7,16.9,30.3,4.4,4.1,30.2,28.3,10.2,...,33.7,51.8,2.4,2.3,9.8,31.6,45.4,2.1,7.7,13.2
Bexar,6.0,4.9,40.8,20.2,28.0,7.9,8.4,26.5,23.1,7.4,...,42.5,42.5,2.7,2.2,10.1,35.1,39.9,3.7,7.7,13.6
Bowie,1.8,5.8,49.0,23.7,19.6,2.4,9.7,38.2,25.9,5.1,...,32.7,46.2,2.2,3.8,15.2,25.3,48.8,2.5,10.5,12.9
Brazoria,5.7,6.7,43.9,19.9,23.9,6.3,7.9,26.9,23.5,8.2,...,33.0,54.3,1.4,2.5,8.8,26.9,52.4,2.3,7.7,10.8
Brazos,2.9,2.4,22.1,9.6,63.0,7.6,8.2,20.1,20.3,4.8,...,50.8,38.9,1.4,1.7,7.1,46.0,37.7,1.1,5.3,9.9
Cameron,7.2,5.3,47.5,24.3,15.7,20.9,15.1,22.9,18.9,5.2,...,36.8,52.7,2.0,2.9,5.5,28.9,46.0,4.6,9.5,11.0
Collin,6.0,5.7,45.2,21.3,21.8,3.0,3.1,16.1,21.4,7.2,...,31.4,58.8,0.7,1.4,7.7,26.6,54.5,1.5,5.7,11.7
